In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re
import glob

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def dfs(tree, st):
    nums = [tree['id']]
    text = [tree['text']]
    print(st + tree['a_attr']['title'])
    st += '    '
    if tree['children'] != None:
        for i in tree['children']:
            t = dfs(i, st)
            nums.extend(t[0])
            text.extend(t[1])
    return nums, text

tree = np.load('data/tree.npy')
nums = []
text = []
for i in tree:
    st = ''
    t = dfs(i, st)
    nums.extend(t[0])
    text.extend(t[1])

d = np.vstack((nums, text))
categories = dict(d.T)

91. Компьютеры
    111. Оргтехника (принтеры, сканеры, факсы)
    106. Аксессуары
    104. Информационные носители и внешние устройства чтения
    101. Периферийные устройства
        327. Клавиатуры, мыши, комплекты
        331. Компьютерная акустика(наушники, колонки) и микрофоны
        332. Мониторы
        335. Прочие периферийные устройства
    94. Комплектующие
        346. Жесткие диски и сетевые накопители
        348. Видеокарты
        352. Процессоры
        401. Материнские платы
        798. Прочие комплектующие
    92. ПК
        245. Моноблоки и стационарные компьютеры
        325. Ноутбуки, нетбуки
    97. Сетевое оборудование
        1198. Настройка сетевого оборудования
    1124. Ремонт компьютеров
        1126. Тест веб-камеры
        1205. Советы по настройке и использованию ПК
            1206. Настройка Windows
    1183. Программное обеспечение
        1204. Драйверы
1186. Эмиграция
    1187. Учеба за границей
    1188. Работа за границей
1190. Сценическое, студи

In [4]:
names = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753]

In [5]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas

parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [6]:
# def getData(tree, names, allUrls, parent_numb, st):
#     dataUrls = []
#     dataText = []
#     dataCategories = []
    
#     number = int(tree['a_attr']['title'].split('.')[0])
#     if number in names and number in files:
#         f = open('data/filterData/categories/{}.txt'.format(number))
#         lines = f.readlines()
#         categUrls = [i.split(', ', 1)[0] for i in lines]
#         categText = [i.split(', ', 1)[1] for i in lines]
#         f.close()
        
#         for i in range(len(categUrls)):
#             if categUrls[i] not in allUrls:
#                 dataUrls.extend([categUrls[i]])
#                 dataText.append([categText[i].replace('\n', '')])
#                 dataCategories.extend([number])

#                 allUrls.add(categUrls[i])
            
#     if len(tree['children']) != 0:
#         for i in tree['children']:
#             a, b, c = getData(i, names, allUrls, number, st)
#             dataUrls.extend(a)
#             dataText.extend(b)
#             dataCategories.extend(c)
    
#     else:
#         if number in names and number in files \
#             and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
#             dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()

#     return dataUrls, dataText, dataCategories

In [7]:
# files = glob.glob('data/filterData/categories/*')
# files = [int(i[27:].split('.')[0]) for i in files]

In [8]:
# ddd = dict()
# for number in files:
#     f = open('data/filterData/categories/{}.txt'.format(number))
#     lines = f.readlines()
#     ddd[number] = len(lines)
#     f.close()

In [9]:
# a = ddd.values()

In [10]:
# import matplotlib.pyplot as plt
# from matplotlib import cm
# %matplotlib inline

In [11]:
# a = list(a)

In [12]:
# b = np.zeros((6))
# for i in a:
#     if i < 10:
#         b[0] += 1
#         continue
#     if i < 50:
#         b[1] += 1
#         continue
#     if i < 100:
#         b[2] += 1
#         continue
#     if i < 200:
#         b[3] += 1
#         continue
#     if i < 500:
#         b[4] += 1
#         continue
#     b[5] += 1

In [13]:
# plt.figure(figsize=(15, 8))
# x = np.arange(b.shape[0])
# w = 1
# h = b/b.max()
# cl = cm.OrRd(h)
# plt.bar(x, b, w, color=cl)
# # plt.xticks(x+0.5, np.arange(b.shape[0]), rotation='vertical')
# plt.xticks(x, ['<10', '10-50', '50-100', '100-200', '200-500', '>500'])
# # plt.axis([0, 5, 0, b.max()+25])
# plt.xlabel('Число доменов в категории', fontsize=14)
# plt.ylabel('Число категорий', fontsize=14)
# plt.savefig('b.eps')
# plt.show()

In [14]:
# allUrls = set()
# dataUrls = []
# dataText = []
# dataCategories = []
# st = '   '
# parent_numb = 1
# for i in tree:
#     a, b, c = getData(i, names, allUrls, parent_numb, st)
#     dataUrls.extend(a)
#     dataText.extend(b)
#     dataCategories.extend(c)

In [15]:
# dataUrls = np.array(dataUrls)
# dataText = np.array(dataText,  dtype='O')
# dataCategories = np.array(dataCategories)

# Train, test

In [16]:
# aaa = list(set(dataCategories))
# dataUrls1 = np.array([])
# dataText1 = np.array([],  dtype='O')
# dataCategories1 = np.array([])
# for i in aaa:
#     if np.sum(dataCategories == i) > 1:
#         dataUrls1 = np.append(dataUrls1, dataUrls[dataCategories == i])
#         dataText1 = np.append(dataText1, dataText[dataCategories == i])
#         dataCategories1 = np.append(dataCategories1, dataCategories[dataCategories == i])
        
# dataUrls = dataUrls1
# dataText = dataText1
# dataCategories = dataCategories1

In [17]:
# dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [18]:
# trainText, testText, trainAns, testAns = \
#     train_test_split(dataText, dataAns, test_size=0.2, stratify=dataCategories)

In [19]:
# np.save('data/filterData/TrainText.npy', trainText)
# np.save('data/filterData/TrainAns.npy', trainAns)
# np.save('data/filterData/TestText.npy', testText)
# np.save('data/filterData/TestAns.npy', testAns)

In [20]:
trainText = np.load('data/filterData/TrainText.npy')
trainAns = np.load('data/filterData/TrainAns.npy')
testText = np.load('data/filterData/TestText.npy')
testAns = np.load('data/filterData/TestAns.npy')

In [21]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

# trainTexts = [i[0] for i in trainText]
# testTexts = [i[0] for i in testText]

trainTexts = trainText
testTexts = testText

In [22]:
tfidfTransformer = TfidfVectorizer()
trainFeatures  = tfidfTransformer.fit_transform(trainTexts)
testFeatures = tfidfTransformer.transform(testTexts)

In [23]:
trainFeatures

<33920x159691 sparse matrix of type '<class 'numpy.float64'>'
	with 11567690 stored elements in Compressed Sparse Row format>

In [24]:
# f = open('myNewData/textAktiv.txt')
# lines = f.readlines()
# f.close()

In [25]:
# lines[0].split(' ', 1)
# lines[1].split(' ', 2)

In [26]:
# regular = np.zeros((len(lines)//2, 4), dtype='O')

In [27]:
# for i in range(len(lines)//2):
#     regular[i, :2] = lines[2*i].split(' ', 1)
#     regular[i, 2:] = lines[2*i+1].split(' ', 2)[1:]

In [28]:
# regularTexts = tfidfTransformer.transform(regular[:,-1])

In [29]:
# from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [30]:
# i = 0
# j = 220
# print(cosine_distances(regularTexts[i], regularTexts[j]))
# print(cosine_similarity(regularTexts[i], regularTexts[j]))

In [31]:
# mn = 2
# for i in range(regularTexts.shape[0]):
#     for j in range(i+1, regularTexts.shape[0]):
# #         mn = min(mn, cosine_similarity(regularTexts[i], regularTexts[j]))
#         if cosine_similarity(regularTexts[i], regularTexts[j]) < 0.6:
#             print(i, j)
#             print(cosine_distances(regularTexts[i], regularTexts[j]))
#             print(cosine_similarity(regularTexts[i], regularTexts[j]))
#             print()

In [32]:
# myDict = {}
# for idx, i in enumerate(regular[:,2]):
#     myDict[i] = myDict.get(i, []) + [idx]

In [33]:
# regular[0]

In [34]:
# newDict = {}
# for idx1, i in enumerate(regular):
#     if i[0] in newDict:
#         if i[2] not in newDict[i[0]]:
#             newDict[i[0]][i[2]] = i[3]
#     else:
#         newDict[i[0]] = {}
#         newDict[i[0]][i[2]] = i[3]

In [35]:
# newDict.keys()

In [36]:
# tt = 916
# a = tfidfTransformer.transform(list(newDict[str(tt)].values()))
# idx = np.array(trainCategory) == tt
# mx = 0
# for i in trainFeatures[idx]:
#     for j in a:
#         if cosine_similarity(j, i) > mx:
#             mx = cosine_similarity(j, i)
#             print(cosine_distances(j, i))
#             print(cosine_similarity(j, i))
#             print()

In [37]:
# from scipy.sparse import vstack

In [38]:
# trainFeatures = vstack((trainFeatures, regularTexts))

In [39]:
# trainCategory.extend(regular[:, 0].astype('float'))

In [40]:
from sklearn import decomposition
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
_start = time.time()
pca = decomposition.TruncatedSVD(n_components=2000)
trainFeatures2 = pca.fit_transform(trainFeatures)
testFeatures2 = pca.transform(testFeatures)
print('LDA fit time:', time.time() - _start)

_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures2, trainCategory)
print('CLF fit time:', time.time() - _time)
predict = clf.predict(testFeatures2)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))
print()

In [11]:
# topics = [500, 1000, 5000, 10000, 30000]
# for topic in topics:
#     print('Number of topics {}'.format(topic))
#     _start = time.time()
#     lda = LatentDirichletAllocation(n_topics=topic, learning_method='batch')
#     trainFeatures2 = lda.fit_transform(trainFeatures)
#     testFeatures2 = lda.transform(testFeatures)
#     print('LDA fit time:', time.time() - _start)

#     _time = time.time()
#     clf = RidgeClassifier()
#     clf.fit(trainFeatures2, trainCategory)
#     print('CLF fit time:', time.time() - _time)
#     predict = clf.predict(testFeatures2)

#     sm = 0
#     print(accuracy_score(testCategory, predict))
#     for i in range(len(testCategory)):
#         if testCategory[i] == predict[i] or \
#             parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
#             str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
#             sm += 1
#     print(sm / len(testCategory))
#     print()

# CLF

In [28]:
_time = time.time()
clf = LogisticRegression()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictRC.npy', predict)

Fit time:  1518.7621824741364
0.599929253626
0.699917462563377


In [47]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictRC.npy', predict)

Fit time:  0.5820331573486328
0.238769013088
0.31812286286994457


Fit time:  1807.1346390247345
0.630821837048
0.7256219785402664

In [53]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictNC.npy', predict)

Fit time:  1.4576611518859863
0.544747081712
0.6610069567268011


In [12]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictKNN1.npy', predict)

Fit time:  0.08399415016174316
0.304209409268
0.3463035019455253


In [11]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictKNN2.npy', predict)

Fit time:  0.07115554809570312
0.559250088433
0.6551114255394411


In [ ]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictSVC.npy', predict)

In [12]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

np.save('data/filterData/predictSVC.npy', predict)

Fit time:  4889.252897500992
0.582360570687
0.6873010258224266


In [13]:
print(len(set(testCategory)))
for i in range(len(testCategory)):
    while(1):
        if parents[str(int(testCategory[i]))] == '0':
            break
        testCategory[i] = int(parents[str(int(testCategory[i]))])
print(len(set(testCategory)))

17
17


In [18]:
predict = np.load('data/filterData/predictRC.npy')
print(len(set(predict)))
for i in range(len(predict)):
    while(1):
        if parents[str(int(predict[i]))] == '0':
            break
        predict[i] = int(parents[str(int(predict[i]))])
print(len(set(predict)))
accuracy_score(testCategory, predict)

348
17


0.86534606768069799

In [19]:
predict = np.load('data/filterData/predictNC.npy')
print(len(set(predict)))
for i in range(len(predict)):
    while(1):
        if parents[str(int(predict[i]))] == '0':
            break
        predict[i] = int(parents[str(int(predict[i]))])
print(len(set(predict)))
accuracy_score(testCategory, predict)

389
17


0.82278033250795901

In [20]:
predict = np.load('data/filterData/predictKNN1.npy')
print(len(set(predict)))
for i in range(len(predict)):
    while(1):
        if parents[str(int(predict[i]))] == '0':
            break
        predict[i] = int(parents[str(int(predict[i]))])
print(len(set(predict)))
accuracy_score(testCategory, predict)

309
17


0.44688126400188655

In [21]:
predict = np.load('data/filterData/predictKNN2.npy')
print(len(set(predict)))
for i in range(len(predict)):
    while(1):
        if parents[str(int(predict[i]))] == '0':
            break
        predict[i] = int(parents[str(int(predict[i]))])
print(len(set(predict)))
accuracy_score(testCategory, predict)

360
17


0.80320716896592381

In [22]:
predict = np.load('data/filterData/predictSVC.npy')
print(len(set(predict)))
for i in range(len(predict)):
    while(1):
        if parents[str(int(predict[i]))] == '0':
            break
        predict[i] = int(parents[str(int(predict[i]))])
print(len(set(predict)))
accuracy_score(testCategory, predict)

263
17


0.83068034429902138

# Error

In [29]:
predict = np.load('data/filterData/predictRC.npy')
categories = np.load('data/categor.npy').item()

In [30]:
predict

array([  510.,   408.,   422., ...,   629.,   147.,  1081.])

In [31]:
# labels = list(set(trainCategory) | set(testCategory))
labels = names
matr = confusion_matrix(testCategory, predict, labels=labels)
for i in range(len(labels)):
#     print('Label:', labels[i], '\tAll:', matr[i].sum(), ' \tTrue:', matr[i, i], '\tMax:',\
#           labels[matr[i].argmax()], '\tMaxS:', matr[i].max(), '\tProc:', matr[i, i]/matr[i].sum())
#     if str(int(labels[i])) in categories:
    srt = matr[i].argsort()[::-1]
    print(categories[str(int(labels[i]))])
    print('All:', matr[i].sum(), ' \tPredict:', matr[i, i])
    for j in range(3):
        if srt[j] != i:
            print('Predict:', matr[i, srt[j]], '\t', categories[str(int(labels[srt[j]]))])
    print()

91. Компьютеры
All: 38  	Predict: 18
Predict: 2 	 154. Товары для туризма
Predict: 2 	 1189. ASUS

111. Оргтехника (принтеры, сканеры, факсы)
All: 54  	Predict: 51
Predict: 1 	 91. Компьютеры
Predict: 1 	 716. Строительное оборудование

106. Аксессуары
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

104. Информационные носители и внешние устройства чтения
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

101. Периферийные устройства
All: 4  	Predict: 0
Predict: 1 	 405. Досуг
Predict: 1 	 433. Apple
Predict: 1 	 1205. Советы по настройке и использованию ПК

327. Клавиатуры, мыши, комплекты
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

331. Компьютерная акустика(наушники, колонки) и микрофоны
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мел

409. Спорт
All: 22  	Predict: 12
Predict: 3 	 1081. Фитнес-клубы
Predict: 2 	 406. Активный отдых

410. Познавательное
All: 9  	Predict: 0
Predict: 1 	 997. Книги
Predict: 1 	 1121. Астрология
Predict: 1 	 1430. Газеты. Новости. Информационные агенства

411. Экскурсии
All: 5  	Predict: 2
Predict: 1 	 753. Отели
Predict: 1 	 732. Туры по России

412. Музеи
All: 2  	Predict: 2
Predict: 0 	 753. Отели
Predict: 0 	 250. Климатическая техника

413. Лектории и мастер-классы
All: 5  	Predict: 0
Predict: 1 	 1030. Творчество и рукоделие
Predict: 1 	 1205. Советы по настройке и использованию ПК
Predict: 1 	 570. Женские журналы

414. Детский досуг
All: 13  	Predict: 4
Predict: 2 	 406. Активный отдых
Predict: 2 	 1030. Творчество и рукоделие

415. Искусство
All: 27  	Predict: 17
Predict: 4 	 1030. Творчество и рукоделие
Predict: 2 	 997. Книги

416. Театры
All: 33  	Predict: 29
Predict: 3 	 428. Билеты
Predict: 1 	 405. Досуг

417. Выставки
All: 13  	Predict: 8
Predict: 2 	 415. Искусство
Predi


627. Mazda
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

628. Mitsubishi
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

635. Honda
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

1291. Suzuki
All: 12  	Predict: 12
Predict: 0 	 753. Отели
Predict: 0 	 249. Бытовая техника

1295. Subaru
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

1299. Datsun
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

1316. Acura
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

1317. Isuzu
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263.

In [42]:
for i in range(len(testCategory)):
    if testCategory[i] != predict[i] and \
        parents[str(int(testCategory[i]))] != str(int(predict[i])) and\
        str(int(predict[i])) not in childrens[str(int(testCategory[i]))]:
            
        print(testAns[i][0])
        print('Test: ', categories[str(int(testCategory[i]))])
        print('Pred: ', categories[str(int(predict[i]))])
        print()

manifik.com.ua
Test:  1229. Одежда (для женщин)
Pred:  668. Одежда

nb.abc.ru
Test:  91. Компьютеры
Pred:  431. Мобильные устройства

lngst.ru
Test:  121. Танки
Pred:  1262. Знакомства

music-festivals.ru
Test:  429. Фестивали
Pred:  1334. Религия

prohunt.kz
Test:  176. Снаряжение для охоты и рыбалки
Pred:  443. Охота

trostnik.su
Test:  708. Деревянное строительство
Pred:  181. Ремонт

pansionat-opora.ru
Test:  426. Отдых на природе
Pred:  753. Отели

narodnayamedicina.com
Test:  1074. Уход
Pred:  570. Женские журналы

lady.webnice.ru
Test:  405. Досуг
Pred:  1081. Фитнес-клубы

ubulbascha.ru
Test:  457. Недвижимость
Pred:  1121. Астрология

antarestech.com
Test:  1183. Программное обеспечение
Pred:  633. Китайские авто

leto-picunda.ru
Test:  752. Съём квартиры
Pred:  1222. Aбхазия

cardguru.ru
Test:  655. Банковские карты
Pred:  642. Финансовые услуги и страхование

nicstyle.ru
Test:  1430. Газеты. Новости. Информационные агенства
Pred:  430. Просмотр спортивной игры

volvo.autopas

Pred:  197. Пол

avto.oglib.ru
Test:  1234. Автолитература
Pred:  997. Книги

balkonidea.ru
Test:  203. Ремонт своими руками
Pred:  227. Окна

kipros.ru
Test:  771. Кипр\ОАЭ\Греция
Pred:  729. Турагенства и туры

webcammictest.com
Test:  1126. Тест веб-камеры
Pred:  474. Фильмы

subsidii.net
Test:  642. Финансовые услуги и страхование
Pred:  641. Журналы и инфопорталы о недвижимости

kancpalitra.ru
Test:  1030. Творчество и рукоделие
Pred:  415. Искусство

zsimonova.ru
Test:  715. Интерьерный дизайн
Pred:  204. Мастер

chessmoscow.ru
Test:  139. Стратегические игры
Pred:  147. Спортивные порталы

excursor.ru
Test:  762. Экскурсии (туризм)
Pred:  411. Экскурсии

seagun.ru
Test:  176. Снаряжение для охоты и рыбалки
Pred:  443. Охота

otdyhv.ru
Test:  743. Информация для туристов
Pred:  458. Коммерческая недвижимость

remont-avtopokraska.ru
Test:  509. Автомобили. Автосервисы
Pred:  624. Nissan

specsplavdv.ru
Test:  194. Стройматериалы
Pred:  722. Водоснабжение и канализация

russia-reg.

Test:  1430. Газеты. Новости. Информационные агенства
Pred:  457. Недвижимость

postroystenu.ru
Test:  708. Деревянное строительство
Pred:  194. Стройматериалы

htc.gsmmoscow.ru
Test:  454. Ремонт мобильных устройств
Pred:  435. Мобильные телефоны

trub-system.ru
Test:  194. Стройматериалы
Pred:  720. Инженерные системы

the-art.ru
Test:  420. Кино
Pred:  415. Искусство

igra-chelovek-pauk.ru
Test:  136. Игры по мультфильмам
Pred:  112. Minecraft

ladaobzor.ru
Test:  1234. Автолитература
Pred:  1247. VAZ

motodisk.ru
Test:  178. Средства передвижения 
Pred:  1219. Мото запчасти и аксессуары

kirpich-safonovo.ru
Test:  1501. Кирпичи, газоблоки, плиты
Pred:  697. Платья

felisatti.pro
Test:  767. Строительный инструмент
Pred:  768. Древо- и металлообрабатывающий инструмент

iphonremont.ru
Test:  433. Apple
Pred:  1189. ASUS

teploprofi.com
Test:  720. Инженерные системы
Pred:  1345. Насосы

mob-acs.ru
Test:  442. Запчасти для мобильных устройств
Pred:  432. Аксессуары для мобильных устро

Test:  90. Всё для спорта и туризма
Pred:  918. Велосипеды

webrealtor.org
Test:  457. Недвижимость
Pred:  431. Мобильные устройства

aromat-for-you.ru
Test:  564. Парфюмерия
Pred:  202. Сантехника

lucecafe.ru
Test:  423. Бары
Pred:  1247. VAZ

harrows-darts.ru
Test:  164. Спортивный инвентарь
Pred:  161. Велоспорт

klanz.ru
Test:  152. Браузерные ММО
Pred:  107. Lineage

market-prog.ru
Test:  508. Автомобили. Ремонт и запчасти
Pred:  622. Японские авто

propellers.ru
Test:  592. Не декоративная косметика для лица
Pred:  563. Косметика

groot.ru
Test:  91. Компьютеры
Pred:  432. Аксессуары для мобильных устройств

rucreditbank.ru
Test:  648. Кредитование
Pred:  1262. Знакомства

bintrade.ru
Test:  799. Бинарные опционы
Pred:  1247. VAZ

snap-tools.ru
Test:  182. Инструмент
Pred:  1345. Насосы

stelsmoto.ru
Test:  510. Автомобили. Запчасти
Pred:  178. Средства передвижения 

avtoteploholod.zakupka.com
Test:  509. Автомобили. Автосервисы
Pred:  250. Климатическая техника

shop-pobedined